### 1. Preliminray

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
import os
from itertools import product
from solver import train_resnet_with_cub
from solver import test_resnet_with_cub

KAGGLE_INPUT_DIR = "cub2002011/CUB_200_2011"
DATA_PATH = os.path.join("/kaggle", "input", KAGGLE_INPUT_DIR)

# NOTE: you need to uncomment the following code if the model is trained in local
# DATA_PATH = os.path.join("data", "CUB_200_2011")

os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

### 2. Grid-Search Optimal Learning Rates

#### 2.1 Set the range of hyper-parameters 

In [17]:
# set hyper-parameters here
num_epochs = [15]
fine_tuning_lrs = [5e-5, 1e-4, 5e-4]
output_lrs = [1e-3, 2e-3, 5e-3, 8e-3, 1e-2]

configurations = list(product(fine_tuning_lrs, output_lrs))

#### 2.2 Train

In [ ]:
best_accs = []

# train with the pretrained model
for config in configurations:
    curr_best_acc = train_resnet_with_cub(DATA_PATH, num_epochs, fine_tuning_lr=config[0], output_lr=config[1])
    best_accs.extend(curr_best_acc)

best_acc = max(best_accs)    
best_fine_tune_lr, best_output_lr = None, None

# write the results into a txt file
with open('best_accuracy_lr.txt', 'w') as f:
    f.write("Configuration        Accuracy\n")
    f.write("=" * 30 + "\n")
    for config, accuracy in zip(configurations, best_accs):
        # find the best config
        if accuracy == best_acc and best_fine_tune_lr is None:
            best_fine_tune_lr, best_output_lr = config
        # write the training record into file 
        f.write("({:>7.5f}, {:>7.5f})   {:>8.6f}\n".format(config[0], config[1], accuracy))

#### 2.3 Inspect the Accuracy

In [5]:
!more .\Output\best_accuracy_lr.txt

Configuration        Accuracy
(0.00005, 0.00100)   0.321540
(0.00005, 0.00200)   0.429928
(0.00005, 0.00500)   0.519330
(0.00005, 0.00800)   0.542630
(0.00005, 0.01000)   0.547463
(0.00010, 0.00100)   0.427684
(0.00010, 0.00200)   0.487401
(0.00010, 0.00500)   0.553158
(0.00010, 0.00800)   0.569900
(0.00010, 0.01000)   0.575078
(0.00050, 0.00100)   0.570590
(0.00050, 0.00200)   0.583362
(0.00050, 0.00500)   0.608215
(0.00050, 0.00800)   0.618053
(0.00050, 0.01000)   0.619261


### 3. Find the Optimal Training Epoch

#### 3.1 Train on a larger epoch

In [ ]:
num_epochs = [15, 30, 45]

# init the configurations
best_accs = []

# train with the pretrained model
curr_best_acc = train_resnet_with_cub(DATA_PATH, num_epochs, best_fine_tune_lr, best_output_lr, save=True)
best_accs.extend(curr_best_acc)
    
# write the results into a txt file
with open('best_accuracy_ep.txt', 'w') as f:
    f.write("Epoch  Accuracy\n")
    f.write("=" * 50 + "\n")
    for num_epoch, accuracy in zip(num_epochs, best_accs):
        f.write("{:>2}     {:>8.6f}\n".format(num_epoch, accuracy))

#### 3.2 Inspect the Accuracy

In [4]:
!more .\Output\best_accuracy_ep.txt

Epoch  Accuracy
15     0.619261
30     0.642734
45     0.647912


### 4. Train with Random Initialized Weights

In [ ]:
curr_best_acc = train_resnet_with_cub(DATA_PATH, num_epochs, best_fine_tune_lr, best_output_lr, pretrain=False)

### 5. Test the trained model

In [8]:
MODEL_PATH = os.path.join("Output", "resnet18_cub.pth")
test_resnet_with_cub(DATA_PATH, MODEL_PATH)

100%|██████████| 91/91 [02:50<00:00,  1.87s/it]

For the class     Black_footed_Albatross     on the CUB dataset, Top-1 accuracy is 0.666667, Top-5 accuracy is 0.933333
For the class        Laysan_Albatross        on the CUB dataset, Top-1 accuracy is 0.600000, Top-5 accuracy is 0.800000
For the class        Sooty_Albatross         on the CUB dataset, Top-1 accuracy is 0.547297, Top-5 accuracy is 1.000000
For the class       Groove_billed_Ani        on the CUB dataset, Top-1 accuracy is 0.600000, Top-5 accuracy is 0.966667
For the class         Crested_Auklet         on the CUB dataset, Top-1 accuracy is 0.793103, Top-5 accuracy is 0.913793
For the class          Least_Auklet          on the CUB dataset, Top-1 accuracy is 0.909091, Top-5 accuracy is 1.000000
For the class        Parakeet_Auklet         on the CUB dataset, Top-1 accuracy is 0.782609, Top-5 accuracy is 0.956522
For the class       Rhinoceros_Auklet        on the CUB dataset, Top-1 accuracy is 0.611111, Top-5 accuracy is 0.888889
For the class        Brewer_Blackbird   